# Παίζοντας Atari με βαθιά ενισχυτική μάθηση

Προσοχή: για να μπορέσει να τρέξει αυτό το notebook στο Kaggle πρέπει να ενεργοποιήσετε το Internet και την GPU για τον πυρήνα σας. Αυτό απαιτεί την επιβεβαίωση του κινητού σας με SMS. Πηγαίντε στο δεξί sidebar στα settings και θα το δείτε. 

Τελικά πρέπει να έχετε αυτή την εικόνα:

![](https://i.imgur.com/Ek5hOIo.jpg)

## Tensorboard και ngrok

Αντικαταστήστε το authentication token σας που παίρνετε από το [ngrok](https://ngrok.com/) στη γραμμή που υποδεικνύεται. Όταν εκτελεστεί το κελί θα σας δώσει το URL όπου μπορείτε να βλέπετε το TensorBoard. Σημειώστε ότι σε περίπτωση επανεκκίνησης του πυρήνα θα πρέπει να ξανατρέξετε το κελί. Η διεύθυνση θα είναι διαφορετική, αλλά τα προηγούμενα στατιστικά σας δεν χάνονται (μέχρι να ανακυκλωθεί ο πυρήνας).

In [79]:
!apt install apt-utils psmisc -y
!killall -9 ngrok
!rm -rf *ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir -p ngrok-dir
!mv ngrok ngrok-dir/
# Στην επόμενη γραμμή βάλτε το δικό σας authentication token από το ngrok
!ngrok-dir/ngrok authtoken 1pHu5BPfSd4DNwnB8O6yywGWZuL_4G27JfRj4EZ5uDca46Gif

LOG_DIR = 'tb_log/'
!mkdir -p LOG_DIR

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('ngrok-dir/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("Wait 10-15 seconds and then click the URL above to open TensorBoard")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
apt-utils is already the newest version (1.6.12ubuntu0.2).
psmisc is already the newest version (23.1-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
ngrok: no process found
--2021-03-15 20:35:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.229.59.32, 52.5.208.118, 52.21.175.83, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.229.59.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.2MB/s    in 0.8s    

2021-03-15 20:35:34 (17.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/

## Εγκατάσταση βιβλιοθήκης και gym

In [80]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
!pip install --upgrade stable_baselines[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3

## Ορισμός παιχνιδιού

Βάλτε εδώ το string που αντιστοιχεί στο παιχνίδι σας. Για την ονοματολογία των περιβαλλόντων:
- v0 vs v4: v0 has repeat_action_probability of 0.25 (meaning 25% of the time the previous action will be used instead of the new action), while v4 has 0 (always follow your issued action)
- Deterministic: a fixed frameskip of 4, while for the env without Deterministic, frameskip is sampled from [2,4]
- There is also NoFrameskip-v4 with no frame skip and no action repeat stochasticity.

Αναλυτικότερα στην εκφώνηση της άσκησης.

In [81]:
atari_env_name='ZaxxonDeterministic-v0'

## Δημιουργία περιβάλλοντος

In [82]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [108]:
# Με τις συναρτήσεις που ακολουθούν κάνουμε την ίδια προεπεξεργασία με την DeepMind

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)

## Εκπαίδευση

Θα εκπαιδεύσουμε ένα δίκτυο deep q-learning (DQN) όπως αυτό της Deepmind. Σημειώστε ότι τα timesteps είναι πολύ λίγα και ότι δεν κάνουμε διερεύνηση στις παραμέτρους του μοντέλου που μπορείτε να βρείτε [εδώ](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters).

In [109]:
import datetime # For filenames while logging

In [116]:
from stable_baselines3 import DQN

model_name='dqn-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

dqn_model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000, learning_rate = 0.0001)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [120]:
from stable_baselines3 import A2C

model_name='a2c-MlpPolicy'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp

a2c_model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=model_log, learning_rate = 0.0001)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [117]:
max_steps=600000
#, eval_env = test_env, log_interval=4
dqn_model.learn(total_timesteps=max_steps)
a2c_model.learn(total_timesteps=max_steps)

Logging to tb_log/dqn-MlpPolicy-20210315-205520/A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.17e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 160      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | -67.1    |
|    learning_rate      | 0.0001   |
|    n_updates          | 99       |
|    policy_loss        | 71.2     |
|    value_loss         | 9.1e+03  |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 1.17e+03  |
|    ep_rew_mean        | 0         |
| time/                 |           |
|    fps                | 154       |
|    iterations         | 200       |
|    time_elapsed       | 6         |
|    total_ti

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 1400     |
|    time_elapsed       | 44       |
|    total_timesteps    | 7000     |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -20.5    |
|    learning_rate      | 0.0001   |
|    n_updates          | 1399     |
|    policy_loss        | -0       |
|    value_loss         | 3.59e+12 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 1500     |
|    time_elapsed       | 48       |
|    total_timesteps    | 7500     |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 2800     |
|    time_elapsed       | 90       |
|    total_timesteps    | 14000    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -3.21    |
|    learning_rate      | 0.0001   |
|    n_updates          | 2799     |
|    policy_loss        | -0       |
|    value_loss         | 5.23e+14 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 2900     |
|    time_elapsed       | 93       |
|    total_timesteps    | 14500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 4100     |
|    time_elapsed       | 132      |
|    total_timesteps    | 20500    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | 0.594    |
|    learning_rate      | 0.0001   |
|    n_updates          | 4099     |
|    policy_loss        | -0       |
|    value_loss         | 9.05e+11 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 4200     |
|    time_elapsed       | 135      |
|    total_timesteps    | 21000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 5400     |
|    time_elapsed       | 173      |
|    total_timesteps    | 27000    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -30.9    |
|    learning_rate      | 0.0001   |
|    n_updates          | 5399     |
|    policy_loss        | -0       |
|    value_loss         | 1.52e+18 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 5500     |
|    time_elapsed       | 176      |
|    total_timesteps    | 27500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 6800     |
|    time_elapsed       | 219      |
|    total_timesteps    | 34000    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -9.59    |
|    learning_rate      | 0.0001   |
|    n_updates          | 6799     |
|    policy_loss        | -0       |
|    value_loss         | 7.54e+18 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 6900     |
|    time_elapsed       | 222      |
|    total_timesteps    | 34500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 8200     |
|    time_elapsed       | 264      |
|    total_timesteps    | 41000    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -104     |
|    learning_rate      | 0.0001   |
|    n_updates          | 8199     |
|    policy_loss        | -0       |
|    value_loss         | 1.41e+20 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 8300     |
|    time_elapsed       | 267      |
|    total_timesteps    | 41500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 9600     |
|    time_elapsed       | 309      |
|    total_timesteps    | 48000    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -1.89    |
|    learning_rate      | 0.0001   |
|    n_updates          | 9599     |
|    policy_loss        | -0       |
|    value_loss         | 4.41e+19 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 9700     |
|    time_elapsed       | 312      |
|    total_timesteps    | 48500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 10900    |
|    time_elapsed       | 351      |
|    total_timesteps    | 54500    |
| train/                |          |
|    entropy_loss       | -0       |
|    explained_variance | -1.01    |
|    learning_rate      | 0.0001   |
|    n_updates          | 10899    |
|    policy_loss        | -0       |
|    value_loss         | 2.4e+19  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.18e+03 |
|    ep_rew_mean        | 0        |
| time/                 |          |
|    fps                | 155      |
|    iterations         | 11000    |
|    time_elapsed       | 354      |
|    total_timesteps    | 55000    |
| train/                |          |
|

## Εκτίμηση απόδοσης

In [118]:
from stable_baselines3.common.evaluation import evaluate_policy

In [121]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"A2C Eval reward: {mean_reward} (+/-{std_reward})")
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"DQN Eval reward: {mean_reward} (+/-{std_reward})")

A2C Eval reward: 0.0 (+/-0.0)
DQN Eval reward: 990.0 (+/-526.2128846769148)


## Σώσιμο εκπαιδευμένου μοντέλου

In [122]:
dqn_model.save("dqn_Zaxxon")
a2c_model.save("a2c_Zaxxon")

Το μοντέλο θα αποθηκευθεί ως zip και μπορείτε να το κατεβάσετε τοπικά από το δεξί sidebar του Kaggle στο "Data" και μετά στο ellipsis menu πάνω στο filename.

## Φόρτωση εκπαιδευμένου μοντέλου

Το Kaggle δεν μας αφήνει να ανεβάσουμε άλλου είδους δεδομένα εκτός από datasets. Για να μπορέσουμε να φέρουμε το εκπαιδευμένο μοντέλο (zip) που κατεβάσαμε ακολουθούμε την εξής διαδικασία:
1. Βάζουμε κάπου στο google drive μας το αρχείο που θέλουμε να μοιραστούμε
2. Κάνουμε δεξί κλικ πάνω του, "Get Link", "Copy Link" και "Done"
3. Κάνουμε paste σε έναν text editor το link. Θα έχει τη μορφή https://drive.google.com/file/d/1COsaNOH8SjbpxxIYc5lOF-QUiUJGU5ZB/view?usp=sharing. Μας ενδιαφέρει μόνο το string που προσδιορίζει μοναδικά το αρχείο μας. Είναι αυτό μετά το "/d/" και πριν το "/view?"
4. Μετατρέπουμε το link ως εξής https://drive.google.com/uc?export=download&id=1COsaNOH8SjbpxxIYc5lOF-QUiUJGU5ZB δηλαδή μετά το "id=" βάζουμε αυτό το string
5. Με το επόμενο κελί κατεβάζουμε το αρχείο και το ονομάζουμε όπως θέλουμε μετά την παράμετρο -O.

Εδώ φαίρνουμε με αυτή την τεχνική ένα μοντέλο Α2C που έχουμε εκπαιδεύσει νωρίτερα

In [85]:
!wget --no-check-certificate 'https://www.dropbox.com/s/he3i5y71w59ephd/dqn_Zaxxon1.zip?dl=1' -O dqn_Zaxxon1.zip

--2021-03-15 20:36:44--  https://www.dropbox.com/s/he3i5y71w59ephd/dqn_Zaxxon1.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/he3i5y71w59ephd/dqn_Zaxxon1.zip [following]
--2021-03-15 20:36:45--  https://www.dropbox.com/s/dl/he3i5y71w59ephd/dqn_Zaxxon1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6861c0eab38070764cfd97b26.dl.dropboxusercontent.com/cd/0/get/BKw4R46Kmky8KOnj6R5SFQicGP_gHzAEGzICo42y5Nt5ClOE0mVua86nkno3eHkXGByNzUkadk9xBaufbdrdlCOzuHIFL4CsnynqhvND44gSFAdv6aeun4tSmXmkxAZbxPtIFveSsTm5zsb_codjt5y4/file?dl=1# [following]
--2021-03-15 20:36:45--  https://uce6861c0eab38070764cfd97b26.dl.dropboxusercontent.com/cd/0/get/BKw4R46Kmky8KOnj6R5SFQicGP_gHzAEGzICo42y5Nt5ClOE0mVua86nkno3eHkXGByNzUkadk9xBau

In [86]:
dqn_model = DQN.load("dqn_Zaxxon", verbose=1)
a2c_model = A2C.load("a2c_Zaxxon", verbose=1)

## Συνέχιση της εκπαίδευσης

In [124]:
new_env = make_atari_env(atari_env_name, n_envs=4)
# Frame-stacking with 4 frames
new_env = VecFrameStack(new_env, n_stack=4)
# Το μοντέλο δεν έχει μαζί του το περιβάλλον, πρέπει να του το αναθέσουμε ξανά.
a2c_model.set_env(new_env)
dqn_model.set_env(new_env)

Wrapping the env in a VecTransposeImage.


In [125]:
max_steps=100000
a2c_model.learn(total_timesteps=max_steps)

Logging to tb_log/a2c-MlpPolicy-20210315-210443/A2C_1


ValueError: could not broadcast input array from shape (4,4,84,84) into shape (1,4,84,84)

In [87]:
mean_reward, std_reward = evaluate_policy(a2c_model, test_env, n_eval_episodes=10)
print(f"A2C Eval reward: {mean_reward} (+/-{std_reward})")
mean_reward, std_reward = evaluate_policy(dqn_model, test_env, n_eval_episodes=10)
print(f"DQN Eval reward: {mean_reward} (+/-{std_reward})")

Eval reward: 1020.0 (+/-620.966987850401)


In [ ]:
dqn_model = DQN.load("dqn_Zaxxon_new", verbose=1)
a2c_model = A2C.load("a2c_Zaxxon_new", verbose=1)

## Καταγραφή video του actual gameplay του πράκτορα

Ωραία τα διαγράμματα, ωραίαο το TensorBoard, αλλά θέλουμε οπωσδήποτε να δούμε τον πράκτορα να παίζει το παιχνίδι!

In [89]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [90]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [91]:
video_folder = '/kaggle/working/videos'

In [92]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [97]:
import numpy as np
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record video_length steps
  env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = env.reset()
  for _ in range(video_length):
    #action, _ = model.predict(obs)
    #θα επιλέξουμε την ενέργεια με τις μεγαλύτερες πιθανότητες
    actions, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = env.step(actions)

  # Close the video recorder
  env.close()

In [105]:
record_video(test_env, dqn_model, video_length=5000, prefix='dqn')

Saving video to  /kaggle/working/videos/dqn-step-0-to-step-5000.mp4


In [104]:
!rm /kaggle/working/videos/dqn_Zaxxon-step-0-to-step-10000.mp4

In [126]:
record_video(test_env, model, video_length=5000, prefix='a2c')

Saving video to  /kaggle/working/videos/a2c-step-0-to-step-5000.mp4


In [107]:
show_videos(video_path = video_folder, prefix='a2c')

Μπορείτε να κατεβάσετε το video με hover πάνω του, κάτω δεξιά στο ellipsis menu και "Download".